In [1]:
from unstructured.partition.pdf import partition_pdf
from langchain.docstore.document import Document
import tempfile

c:\Software_Tools\Anaconda\envs\ADRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def pdf_processor1(bytes):
        with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as tmp:
            tmp.write(bytes)
            tmp.flush()                  
            tmp_path = tmp.name
            raw_pdf_elements=partition_pdf(
            filename=tmp_path,         # mandatory
            strategy="hi_res",                                 # mandatory to use ``hi_res`` strategy
            extract_images_in_pdf=True,                       # mandatory to set as ``True``
            extract_image_block_types=["Image", "Table"],          # optional
            extract_image_block_to_payload=False)              # optiona
                        
        return raw_pdf_elements 

In [3]:
file_path = r"startupai-financial-report.pdf"
with open(file_path, "rb") as f:
    bytes = f.read()

In [3]:
file_path = r"startupai-financial-report.pdf"
output_dir = r"all_images"


In [4]:
raw_pdf_elements = pdf_processor1(bytes)

In [5]:
raw_pdf_elements

In [6]:
def extract_text_elements(raw_pdf_elements):
    Header, Footer, Title , NarrativeText, Text , ListItem , Img , Tables =  [], [], [],  [], [], [], [], []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Header" in str(type(element)):
             Header.append(str(element))
        elif "unstructured.documents.elements.Footer" in str(type(element)):
             Footer.append(str(element))
        elif "unstructured.documents.elements.Title" in str(type(element)):
             Title.append(str(element))
        elif "unstructured.documents.elements.NarrativeText" in str(type(element)):
             NarrativeText.append(str(element))
        elif "unstructured.documents.elements.Text" in str(type(element)):
             Text.append(str(element))
        elif "unstructured.documents.elements.ListItem" in str(type(element)):
             ListItem.append(str(element))
        elif "unstructured.documents.elements.Image" in str(type(element)):
             Img.append(str(element))
        elif "unstructured.documents.elements.Table" in str(type(element)):
             Tables.append(str(element))
        elif 'unstructured.documents.elements.CompositeElement' in str(type(element)):
             Text.append(str(element))
    return Header, Footer, Title , NarrativeText, Text , ListItem , Img , Tables

In [7]:
Header, Footer, Title , NarrativeText, Text , ListItem , Img , Tables = extract_text_elements(raw_pdf_elements)

In [9]:
Text


['2020 2021 2022 2023 2024']

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
api_endpoint = os.getenv("ASTRA_DB_API_ENDPOINT")
app_token = os.getenv("ASTRA_DB_APPLICATION_TOKEN")

In [3]:
from langchain_astradb import AstraDBVectorStore

c:\Software_Tools\Anaconda\envs\openAI\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain_astradb import AstraDBVectorStore
from langchain.schema import Document
import os
import torch
import clip
from PIL import Image
from langchain.embeddings.base import Embeddings
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="clip")


device = "cuda" if torch.cuda.is_available() else "cpu"
# Load and save weights locally
model, preprocess = clip.load(
    "ViT-B/32",
    download_root="C:/models/clip_weights"  # <-- this folder will store weights
)

class CLIPEmbeddings(Embeddings):
    def __init__(self, model, preprocess, device):
        self.model = model
        self.preprocess = preprocess
        self.device = device

    def embed_query(self, text: str = None, **kwargs):
        if text is None:
            raise ValueError("No text provided to embed_query")

        if os.path.exists(text) and text.lower().endswith(('.png', '.jpg', '.jpeg')):
            image = Image.open(text).convert("RGB")
            return self.embed_image(image)
        else:
            return self.embed_text(text)

    def embed_documents(self, texts_or_paths):
        """
        Handle list of text or image paths
        """
        embeddings = []
        for item in texts_or_paths:
            embeddings.append(self.embed_query(item))
        return embeddings

    def embed_text(self, text: str):
        """Embed text only"""
        text_features = clip.tokenize([text], truncate=True).to(self.device)
        with torch.no_grad():
            emb = self.model.encode_text(text_features).cpu().numpy()[0]
        return emb.tolist()

    def embed_image(self, image: Image.Image):
        """Embed a PIL image"""
        image_tensor = self.preprocess(image).unsqueeze(0).to(self.device)
        with torch.no_grad():
            emb = self.model.encode_image(image_tensor).cpu().numpy()[0]
        return emb.tolist()


# Initialize embeddings
clip_embeddings = CLIPEmbeddings(model, preprocess, device)

In [5]:
vector_store = AstraDBVectorStore(
    collection_name = "RAG",
    embedding = clip_embeddings,
    api_endpoint = api_endpoint,
    token = app_token,
    pre_delete_collection=False,
    metadata_indexing_include=["source", "type", "doc_id"]
)

In [10]:
astra_index = {'Collection_name' : "Chetan"}

In [11]:
user_collections = {}

In [12]:
user_collections["user_id"] = astra_index

In [15]:
user_collections

{}

In [14]:
del user_collections['user_id']

In [ ]:
vector_store.collection_name

AsyncAstraDBCollection[astra_db="AsyncAstraDB[endpoint="https://59ec24d5-66e0-4139-a964-ac6b4c4c6ca0-us-east-2.apps.astra.datastax.com", keyspace="default_keyspace"]", collection_name="RAG"]

In [19]:
from astrapy import DataAPIClient

In [20]:
database = DataAPIClient(app_token).get_database(api_endpoint)

In [22]:
database.list_collection_names()

['Multimodal_RAG']

In [1]:
import shutil
import os

def delete_output_dir(output_dir):
    """
    Deletes the specified output directory (and all files/subdirs within it).
    """
    try:
        if os.path.exists(output_dir):
            shutil.rmtree(output_dir)
            print(f"✅ Deleted directory: {output_dir}")
        else:
            print(f"⚠️ Directory not found: {output_dir}")
    except Exception as e:
        print(f"❌ Failed to delete directory {output_dir}: {e}")


In [4]:
path = r"D:\GEN AI\GEN_AI_MASTERS_Bappy\Langchain\Advanced_prod_RAG\all_images\4\2025-11-10_16-01-30"
delete_output_dir(path)

✅ Deleted directory: D:\GEN AI\GEN_AI_MASTERS_Bappy\Langchain\Advanced_prod_RAG\all_images\4\2025-11-10_16-01-30


In [5]:
from pathlib import Path
from langchain_core.documents import Document
from src.logger_config import log
import os, time

In [9]:
def get_doc_image_dir(file_name,user_id): 
    doc_name = Path(file_name).stem # give only file name without etension
    base_output_dir = "all_images"
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = os.path.join(base_output_dir, user_id, timestamp, doc_name + "_images")
    os.makedirs(output_dir,exist_ok=True)
    return output_dir

In [10]:
file_name = r"requirements.txt"
output_dir = get_doc_image_dir(file_name, "123")

In [8]:
delete_output_dir(output_dir)

✅ Deleted directory: all_images\123\2025-11-11_13-19-55\requirements_images


In [13]:
output_dir

'all_images\\123\\2025-11-11_13-21-51\\requirements_images'

In [ ]:
import os
import shutil

def delete_user_folder(user_id, base_dir="all_images"):
    """
    Deletes the entire user folder (e.g., all_images/<user_id>) safely.
    Example:
        delete_user_folder("123")
    """
    try:
        user_dir = os.path.join(base_dir, str(user_id))

        if not os.path.exists(user_dir):
            print(f"⚠️ No such user folder: {user_dir}")
            return

        # Safety guard — ensure we're really deleting inside all_images
        if os.path.basename(os.path.dirname(user_dir)) != os.path.basename(base_dir):
            print(f"⚠️ Unsafe path detected: {user_dir}")
            return

        shutil.rmtree(user_dir)
        print(f"✅ Deleted user folder: {user_dir}")

    except Exception as e:
        print(f"❌ Failed to delete user folder {user_id}: {e}")


In [ ]:
os.path.

In [17]:
base_dir = "all_images"
user_id = "23"
path = os.path.join(base_dir,user_id)

In [26]:
path

'all_images\\23'

In [19]:
os.path.dirname(path)


'all_images'

In [28]:
os.path.basename("all_images")

'all_images'

In [ ]:
os.path.basename(os.path.dirname('all_images\\23')) != os.path.basename("all_images")

False

In [25]:
os.path.basename(base_dir)

'all_images'

In [24]:
os.path.basename("all_images")

'all_images'

In [ ]:
!pip show streamlit

: 

In [2]:
user_info = {'iss': 'https://accounts.google.com', 'azp': '225813822699-k534tbs58olpeuovurvpb7oks395u0hc.apps.googleusercontent.com', 'aud': '225813822699-k534tbs58olpeuovurvpb7oks395u0hc.apps.googleusercontent.com', 'sub': '113076068933443359029', 'email': 'chetanfern@gmail.com', 'email_verified': True, 'at_hash': 'buvYfbfa23OcdlhtT-fbtg', 'nonce': 'CLvyNgjNJcpq29Njdnj0', 'name': 'Chetan Fenandis', 'picture': 'https://lh3.googleusercontent.com/a/ACg8ocJ-xg5rds6ARMCqO9xmE6Ye9uBuodfYqaoApGaBVledqlfrRw=s96-c', 'given_name': 'Chetan', 'family_name': 'Fenandis', 'iat': 1762970767, 'exp': 1762974367}

In [ ]:
from itsdangerous import URLSafeSerializer

serializer = URLSafeSerializer("your_secret_key")

In [ ]:
Internally, itsdangerous uses:

HMAC (Hash-based Message Authentication Code)

with SHA256 hashing

salted using your secret key

In [ ]:
itsdangerous is a Python library that helps you safely encode data (like user info) into strings that you can send over URLs, cookies, or APIs — without letting attackers modify them.

In [ ]:
signature = HMAC_SHA256(secret_key, data)

In [23]:
signed_user = serializer.dumps(dict(user_info))

In [24]:
signed_user

'.eJytkN1LwzAUxf8V6bNdP9KmzUBwGyoM3WCIsKeRpmmbrU3SfNRN8X83GxYm-Ojr-d1z7rn302Nae1OvMUbqaRBgQoTlRk9qIeqWTojovFsPf0g3E8dpHoE8jiFC_iEFiSl0motWUisGqwZZZOKgAUpt2JAJlnJMsZoqIrih3IyBtvzfQG0LFxhFIMxgCHMEQJIAkKIwRo7SDrPWcdJQg3lFFb-vz9KP-YJ3A1WsYtQVM8pS19HsGqwbZyvssK2KCsdgTcq2Ma9-VZjaGbnghLqBxfNwWtX71ZLIPkarfcn34Rnj7kIvW28eKce8ZNoByYixil79vW3A37cFOJgt6lyQpX-sU1VqONu8LPo1OnYPcEuRnVtRVtsei5l8wvO3lpZ9W6nN-51G0CduWc0Gyne_uji1wh1rT6N81Y1h402jDMYoc7_M3HeOchQSALOvbzeLuP8.WDi-hEpNpeti2QiBJvGUh1E8iq0'

In [25]:

serializer = URLSafeSerializer("your_secret_key")

In [30]:
decoded = {
  "user": {
    "iss": "https://accounts.google.com",
    "azp": "225813822699-k534tbs58olpeuovurvpb7oks395u0hc.apps.googleusercontent.com",
    "aud": "225813822699-k534tbs58olpeuovurvpb7oks395u0hc.apps.googleusercontent.com",
    "sub": "113076068933443359029",
    "email": "chetanfern@gmail.com",
    "at_hash": "LPSCeQdAms85fMtrjtzv8A",
    "nonce": "t0QAEZmyN3jUcJ05BxGJ",
    "name": "Chetan Fenandis",
    "picture": "https://lh3.googleusercontent.com/a/ACg8ocJ-xg5rds6ARMCqO9xmE6Ye9uBuodfYqaoApGaBVledqlfrRw=s96-c",
    "given_name": "Chetan",
    "family_name": "Fenandis",
    "iat": 1763056286,
    "exp": 1763059886
  },
  "exp": 1763058086,
  "iat": 1763056286
}

In [40]:
decoded.user

AttributeError: 'dict' object has no attribute 'user'

In [41]:

headers = {"Authorization": f"Bearer {113076068933443359029}"}

In [42]:
token = headers.split(" ")[1]

AttributeError: 'dict' object has no attribute 'split'

In [ ]:
Authenticated user: chetanfern@gmail.com (113076068933443359029)

In [37]:
user_info

{'iss': 'https://accounts.google.com',
 'azp': '225813822699-k534tbs58olpeuovurvpb7oks395u0hc.apps.googleusercontent.com',
 'aud': '225813822699-k534tbs58olpeuovurvpb7oks395u0hc.apps.googleusercontent.com',
 'sub': '113076068933443359029',
 'email': 'chetanfern@gmail.com',
 'at_hash': 'LPSCeQdAms85fMtrjtzv8A',
 'nonce': 't0QAEZmyN3jUcJ05BxGJ',
 'name': 'Chetan Fenandis',
 'picture': 'https://lh3.googleusercontent.com/a/ACg8ocJ-xg5rds6ARMCqO9xmE6Ye9uBuodfYqaoApGaBVledqlfrRw=s96-c',
 'given_name': 'Chetan',
 'family_name': 'Fenandis',
 'iat': 1763056286,
 'exp': 1763059886}

In [ ]:
import requests
import streamlit as st

API_URL = "https://your-fastapi-backend.com"
SAVED_BOOKS_URL = f"{API_URL}/user-books/"

st.title("📚 My Saved Books")

# Assume user is already authenticated and token stored in session
access_token = st.session_state.get("access_token")
user_id = st.session_state.get("user_id")

if not access_token or not user_id:
    st.error("Please log in to view saved books.")
    st.stop()

headers = {"Authorization": f"Bearer {access_token}"}
response = requests.get(SAVED_BOOKS_URL, params={"user_id": user_id}, headers=headers)

if response.status_code != 200:
    st.error("Failed to load saved books.")
    st.stop()

saved_books = response.json()

for book in saved_books:
    st.subheader(book["title"])
    st.write(f"**Author(s):** {book['authors']}")
    st.write(f"**Published:** {book.get('published_date', 'N/A')}")
    st.markdown("---")

AttributeError: 'dict' object has no attribute 'name'